In [4]:
def pretreatment(wav):  
    # 전처리 소수점 0.3 이하까지만
    y=[]
    for i in range(len(wav)):
        y.append(round(wav[i],3))
        
    # 0.02 이하는 0으로    
    t=[]
    for i in range(len(y)):
        if y[i]>=0.02 or  y[i]<=-0.02:
            t.append(y[i])
        else:
            y[i]=0
            t.append(y[i])
    
    # 파장 나누기
    re_time =[] # 파장 나눈 위치 저장 리스트
    count = 0 # 0의 개수 세기
    # and len(re_time)%2==0
    for i in range(len(t)):
        if t[i]==0:
            count+=1
        elif t[i]!=0 and count>=500:
            re_time.append(i)
            count=0
        else:
            count=0
    
    # 파장 길이 구하기
    l_list=[] # 파장 길이 저장 리스트 (x)
    l_count=0 # 0의 개수 카운트
    for i in range(len(re_time)):
        if i == len(re_time)-1:
            for j in range(len(t[re_time[i]:])):
                a = re_time[i]
                if t[a+j]!=0:
                    l_count+=1
            l_list.append(l_count)
            l_count=0
        else:
            for j in range(len(t[re_time[i]:re_time[i+1]])):
                a = re_time[i]
                if t[a+j]!=0:
                    l_count+=1
            l_list.append(l_count)
            l_count=0
            l_list.append

    # 두개로 나누기
    s_all=sum(l_list)/2
    b=0 # 각 파장 길이 중첩 변수
    c=0 # 몇번째 파장까지 나누는지 결정 변수
    for i in range(len(l_list)):
        b+=l_list[i]
        if len(l_list)==2:
            c=i+1
            break
    
        elif b >=s_all:
            c=i+1
            break
            
    global wav_01
    global wav_02
    # 각 나눈 파장 저장        
    wav_01 = wav[:re_time[c]]
    wav_02 = wav[re_time[c]:]



In [6]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display
import pandas as pd

WAV_DIR = "./wav/wav/"

x = []
y = []
# os.listdir(TRAIN_DIR) : train 폴더로부터 wav 파일을 하나씩 읽어온다

# train
for fname in os.listdir(WAV_DIR):
    try:
        # 읽은 파일이 wav 파일이 아닌 경우 다음으로 넘어감
        if ".wav" not in fname:
            pass
        
        # 라벨 데이터를 저장
        # text는 파일 명?에서 따올 것
        text =fname[:len(fname)-4]
        ts = text.split("_")
        print(ts)
        y.append(ts[0])
        y.append(ts[1])
        
        # wav 파일 데이터를 읽어온다
        wav, sr = librosa.load(WAV_DIR+fname,sr=16000)
        
        pretreatment(wav)
        
        # mfcc 전환
        mfcc_01 = librosa.feature.mfcc(wav_01)
        mfcc_01 = librosa.feature.mfcc(wav_01, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
        mfcc_02 = librosa.feature.mfcc(wav_02)
        mfcc_02 = librosa.feature.mfcc(wav_02, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)      
        
        zero_pad_01 = np.zeros((100,400-len(mfcc_01[0])))
        mfcc_01 = np.hstack((mfcc_01,zero_pad_01)) 
        
        zero_pad_02 = np.zeros((100,400-len(mfcc_02[0])))
        mfcc_02 = np.hstack((mfcc_02,zero_pad_02)) 
        
        print(mfcc_01.shape,mfcc_02.shape)
        
        # wav 파일을 동일한 길이로 잘라서 특성 데이터로 저장
        # wav 데이터의 길이를 30000으로 설정
        #mfcc_01 = np.expand_dims(mfcc_01, -1)
        #mfcc_02 = np.expand_dims(mfcc_02, -1)
        
        x.append(mfcc_01)
        x.append(mfcc_02)
    except:
        print("파일 읽기 오류")
        # 다음으로 넘어간다
        raise

['고생1', '했어1']
(100, 400) (100, 400)
['고생2', '했어2']
(100, 400) (100, 400)
['고생3', '했어3']
(100, 400) (100, 400)
['사랑1', '해요1']
(100, 400) (100, 400)
['사랑2', '해요2']
(100, 400) (100, 400)
['사랑3', '해요3']
(100, 400) (100, 400)
['응원1', '할게요1']
(100, 400) (100, 400)
['응원2', '할게요2']
(100, 400) (100, 400)
['응원3', '할게요3']
(100, 400) (100, 400)
['축하1', '해요4']
(100, 400) (100, 400)
['축하2', '해요5']
(100, 400) (100, 400)
['축하3', '해요6']
(100, 400) (100, 400)
['행복1', '해요7']
(100, 400) (100, 400)
['행복2', '해요8']
(100, 400) (100, 400)
['행복3', '해요9']
(100, 400) (100, 400)


In [7]:
x = np.array(x)
x_ex = np.expand_dims(x,-1)

y = np.array(y)
y_en = pd.get_dummies(y)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()

#CNN 층 : 이미지 데이터로부터 특징을 추출하는 전처리 (특성추출기)
#일반적으로 Conv2D는 2개나 3개와 MaxPooling2D 1개로 레이어를 구성
#컨볼루션 레이어는 데이터의 크기가 10x10 근처의 크기가 될때까지 반복
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(100,400,1), padding="same", activation="relu"))

model.add(Conv2D(filters=64, kernel_size=(3, 3),  padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3),  padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

    # DNN에 넣어주기 위해서는 다차원 데이터를 1차원으로 펴주어야 한다
model.add(Flatten())

    # 은닉층 1차원만 들어갈 수 있어서 Flatten()을 사용해 1차원으로 펴준다.
model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
    
    # 출력층
model.add(Dense(len(y_en), activation="softmax"))
model.add(BatchNormalization())
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 400, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 400, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 200, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 200, 64)       36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 200, 128)      73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 100, 128)      0         
_________________________________________________________________
flatten (Flatten)            (None, 320000)            0

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
#모델을 저장할 폴더
MODEL_DIR = './model4/'

#폴더가 없다면 생성
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
#loss값에 따라서 파일명을 지을거임
modelpath = MODEL_DIR + '{epoch:02d}-{val_loss:.4f}-{accuracy:.4f}.hdf5'

#모델이 오차범위를 수정하면서 정확도가 높아지면 파일을 갱신할거임
mc = ModelCheckpoint(filepath = modelpath, #filepath : 저장할 파일경로
               monitor = 'accuracy', verbose = 1, #monitor : 어떤걸 모니터(체크)할건가 ,   val_loss: 테스트오차, val_acc : 테스트 정확도, loss : 훈련오차 // verboss : 진행내용 출력여부 1출력 0미출력
               save_best_only = 'True') #save_best_only : 가장높은 수치반영 True : 이전정확도보다 높으면 저장 // False : 낮으면 저장

h= model.fit(x_ex, y_en, validation_split = 0.3, epochs=500, callbacks=[mc])

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 10.5983 - accuracy: 0.2857
Epoch 00001: accuracy improved from -inf to 0.28571, saving model to ./model4\01-4.5201-0.2857.hdf5
1/1 [==============================] - 3s 3s/step - loss: 10.5983 - accuracy: 0.2857 - val_loss: 4.5201 - val_accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - ETA: 0s - loss: 10.4156 - accuracy: 0.2857
Epoch 00002: accuracy did not improve from 0.28571
1/1 [==============================] - 0s 183ms/step - loss: 10.4156 - accuracy: 0.2857 - val_loss: 4.5036 - val_accuracy: 0.0000e+00
Epoch 3/500
1/1 [==============================] - ETA: 0s - loss: 8.7213 - accuracy: 0.2857
Epoch 00003: accuracy did not improve from 0.28571
1/1 [==============================] - 0s 201ms/step - loss: 8.7213 - accuracy: 0.2857 - val_loss: 4.4918 - val_accuracy: 0.0000e+00
Epoch 4/500
1/1 [==============================] - ETA: 0s - loss: 8.9323 - accuracy: 0.2381
Epoch 00004: accuracy did 

Epoch 30/500
1/1 [==============================] - ETA: 0s - loss: 13.1051 - accuracy: 0.4286
Epoch 00030: accuracy did not improve from 0.42857
1/1 [==============================] - 0s 190ms/step - loss: 13.1051 - accuracy: 0.4286 - val_loss: 4.3673 - val_accuracy: 0.0000e+00
Epoch 31/500
1/1 [==============================] - ETA: 0s - loss: 12.3342 - accuracy: 0.3333
Epoch 00031: accuracy did not improve from 0.42857
1/1 [==============================] - 0s 175ms/step - loss: 12.3342 - accuracy: 0.3333 - val_loss: 4.3832 - val_accuracy: 0.0000e+00
Epoch 32/500
1/1 [==============================] - ETA: 0s - loss: 10.7793 - accuracy: 0.4286
Epoch 00032: accuracy did not improve from 0.42857
1/1 [==============================] - 0s 163ms/step - loss: 10.7793 - accuracy: 0.4286 - val_loss: 4.3916 - val_accuracy: 0.0000e+00
Epoch 33/500
1/1 [==============================] - ETA: 0s - loss: 12.3255 - accuracy: 0.3333
Epoch 00033: accuracy did not improve from 0.42857
1/1 [=========

1/1 [==============================] - ETA: 0s - loss: 13.0480 - accuracy: 0.2857
Epoch 00059: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 162ms/step - loss: 13.0480 - accuracy: 0.2857 - val_loss: 5.4670 - val_accuracy: 0.0000e+00
Epoch 60/500
1/1 [==============================] - ETA: 0s - loss: 12.2872 - accuracy: 0.3810
Epoch 00060: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 155ms/step - loss: 12.2872 - accuracy: 0.3810 - val_loss: 5.4775 - val_accuracy: 0.0000e+00
Epoch 61/500
1/1 [==============================] - ETA: 0s - loss: 12.2805 - accuracy: 0.3333
Epoch 00061: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 157ms/step - loss: 12.2805 - accuracy: 0.3333 - val_loss: 5.4907 - val_accuracy: 0.0000e+00
Epoch 62/500
1/1 [==============================] - ETA: 0s - loss: 12.2821 - accuracy: 0.3810
Epoch 00062: accuracy did not improve from 0.47619
1/1 [======================

Epoch 89/500
1/1 [==============================] - ETA: 0s - loss: 11.1863 - accuracy: 0.3810
Epoch 00089: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 168ms/step - loss: 11.1863 - accuracy: 0.3810 - val_loss: 7.7674 - val_accuracy: 0.1111
Epoch 90/500
1/1 [==============================] - ETA: 0s - loss: 11.6537 - accuracy: 0.4286
Epoch 00090: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 171ms/step - loss: 11.6537 - accuracy: 0.4286 - val_loss: 7.7685 - val_accuracy: 0.1111
Epoch 91/500
1/1 [==============================] - ETA: 0s - loss: 11.9462 - accuracy: 0.4286
Epoch 00091: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 152ms/step - loss: 11.9462 - accuracy: 0.4286 - val_loss: 7.7751 - val_accuracy: 0.1111
Epoch 92/500
1/1 [==============================] - ETA: 0s - loss: 11.0329 - accuracy: 0.4286
Epoch 00092: accuracy did not improve from 0.47619
1/1 [=====================

Epoch 119/500
1/1 [==============================] - ETA: 0s - loss: 11.6417 - accuracy: 0.4762
Epoch 00119: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 166ms/step - loss: 11.6417 - accuracy: 0.4762 - val_loss: 6.6853 - val_accuracy: 0.1111
Epoch 120/500
1/1 [==============================] - ETA: 0s - loss: 11.0121 - accuracy: 0.4762
Epoch 00120: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 164ms/step - loss: 11.0121 - accuracy: 0.4762 - val_loss: 6.7253 - val_accuracy: 0.1111
Epoch 121/500
1/1 [==============================] - ETA: 0s - loss: 10.0979 - accuracy: 0.4762
Epoch 00121: accuracy did not improve from 0.47619
1/1 [==============================] - 0s 166ms/step - loss: 10.0979 - accuracy: 0.4762 - val_loss: 6.7676 - val_accuracy: 0.1111
Epoch 122/500
1/1 [==============================] - ETA: 0s - loss: 10.8763 - accuracy: 0.4762
Epoch 00122: accuracy did not improve from 0.47619
1/1 [=================

...

Epoch 382/500
1/1 [==============================] - ETA: 0s - loss: 3.6002 - accuracy: 0.3810
Epoch 00382: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 172ms/step - loss: 3.6002 - accuracy: 0.3810 - val_loss: 9.4371 - val_accuracy: 0.1111
Epoch 383/500
1/1 [==============================] - ETA: 0s - loss: 4.4953 - accuracy: 0.3810
Epoch 00383: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 182ms/step - loss: 4.4953 - accuracy: 0.3810 - val_loss: 9.4661 - val_accuracy: 0.1111
Epoch 384/500
1/1 [==============================] - ETA: 0s - loss: 5.1169 - accuracy: 0.3810
Epoch 00384: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 167ms/step - loss: 5.1169 - accuracy: 0.3810 - val_loss: 9.4797 - val_accuracy: 0.1111
Epoch 385/500
1/1 [==============================] - ETA: 0s - loss: 4.2310 - accuracy: 0.4762
Epoch 00385: accuracy did not improve from 0.76190
1/1 [========================

Epoch 412/500
1/1 [==============================] - ETA: 0s - loss: 4.4243 - accuracy: 0.4286
Epoch 00412: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 162ms/step - loss: 4.4243 - accuracy: 0.4286 - val_loss: 11.4870 - val_accuracy: 0.0000e+00
Epoch 413/500
1/1 [==============================] - ETA: 0s - loss: 6.7208 - accuracy: 0.5714
Epoch 00413: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 154ms/step - loss: 6.7208 - accuracy: 0.5714 - val_loss: 11.5699 - val_accuracy: 0.0000e+00
Epoch 414/500
1/1 [==============================] - ETA: 0s - loss: 5.9220 - accuracy: 0.6190
Epoch 00414: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 173ms/step - loss: 5.9220 - accuracy: 0.6190 - val_loss: 11.6182 - val_accuracy: 0.0000e+00
Epoch 415/500
1/1 [==============================] - ETA: 0s - loss: 4.3913 - accuracy: 0.5714
Epoch 00415: accuracy did not improve from 0.76190
1/1 [=========

1/1 [==============================] - ETA: 0s - loss: 12.2805 - accuracy: 0.5238
Epoch 00441: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 188ms/step - loss: 12.2805 - accuracy: 0.5238 - val_loss: 8.2941 - val_accuracy: 0.0000e+00
Epoch 442/500
1/1 [==============================] - ETA: 0s - loss: 13.0480 - accuracy: 0.5714
Epoch 00442: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 163ms/step - loss: 13.0480 - accuracy: 0.5714 - val_loss: 6.4955 - val_accuracy: 0.0000e+00
Epoch 443/500
1/1 [==============================] - ETA: 0s - loss: 13.8258 - accuracy: 0.4762
Epoch 00443: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 181ms/step - loss: 13.8258 - accuracy: 0.4762 - val_loss: 6.4998 - val_accuracy: 0.0000e+00
Epoch 444/500
1/1 [==============================] - ETA: 0s - loss: 13.0554 - accuracy: 0.4762
Epoch 00444: accuracy did not improve from 0.76190
1/1 [===================

1/1 [==============================] - ETA: 0s - loss: 13.9408 - accuracy: 0.5238
Epoch 00470: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 172ms/step - loss: 13.9408 - accuracy: 0.5238 - val_loss: 4.7363 - val_accuracy: 0.0000e+00
Epoch 471/500
1/1 [==============================] - ETA: 0s - loss: 12.3973 - accuracy: 0.5238
Epoch 00471: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 199ms/step - loss: 12.3973 - accuracy: 0.5238 - val_loss: 4.7854 - val_accuracy: 0.0000e+00
Epoch 472/500
1/1 [==============================] - ETA: 0s - loss: 11.6391 - accuracy: 0.3810
Epoch 00472: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 170ms/step - loss: 11.6391 - accuracy: 0.3810 - val_loss: 4.8257 - val_accuracy: 0.0000e+00
Epoch 473/500
1/1 [==============================] - ETA: 0s - loss: 13.2036 - accuracy: 0.4286
Epoch 00473: accuracy did not improve from 0.76190
1/1 [===================

1/1 [==============================] - ETA: 0s - loss: 13.0480 - accuracy: 0.5238
Epoch 00499: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 166ms/step - loss: 13.0480 - accuracy: 0.5238 - val_loss: 10.0716 - val_accuracy: 0.0000e+00
Epoch 500/500
1/1 [==============================] - ETA: 0s - loss: 10.8931 - accuracy: 0.4762
Epoch 00500: accuracy did not improve from 0.76190
1/1 [==============================] - 0s 167ms/step - loss: 10.8931 - accuracy: 0.4762 - val_loss: 10.0893 - val_accuracy: 0.0000e+00


In [14]:
TEST_REAL_DIR = "./wav/test_real/"

X_real = []

for fname in os.listdir(TEST_REAL_DIR):
    try:
        # 읽은 파일이 wav 파일이 아닌 경우 다음으로 넘어감
        if ".wav" not in fname:
            pass
        
        
        ####테스트 주석#####
        
        # 라벨 데이터를 저장
        # text는 파일 명?에서 따올 것
        #text =fname[:len(fname)-4]
        #ts = text.split("_")
        #print(ts)
        #y_test.append(ts[0])
        #y_test.append(ts[1])
        
        # wav 파일 데이터를 읽어온다
        wav, sr = librosa.load(TEST_REAL_DIR+fname,sr=16000)
        
        pretreatment(wav)
        
        # mfcc 전환
        mfcc_01 = librosa.feature.mfcc(wav_01)
        mfcc_01 = librosa.feature.mfcc(wav_01, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
        mfcc_02 = librosa.feature.mfcc(wav_02)
        mfcc_02 = librosa.feature.mfcc(wav_02, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)      
        
        zero_pad_01 = np.zeros((100,400-len(mfcc_01[0])))
        mfcc_01 = np.hstack((mfcc_01,zero_pad_01)) 
        
        zero_pad_02 = np.zeros((100,400-len(mfcc_02[0])))
        mfcc_02 = np.hstack((mfcc_02,zero_pad_02)) 
        
        print(mfcc_01.shape,mfcc_02.shape)
        
        # wav 파일을 동일한 길이로 잘라서 특성 데이터로 저장
        # wav 데이터의 길이를 30000으로 설정
        #mfcc_01 = np.expand_dims(mfcc_01, -1)
        #mfcc_02 = np.expand_dims(mfcc_02, -1)
        
        X_real.append(mfcc_01)
        X_real.append(mfcc_02)
    except:
        print("파일 읽기 오류")
        # 다음으로 넘어간다

(100, 400) (100, 400)


In [15]:
X_real = np.array(X_real)

X_real_ex = np.expand_dims(X_real, -1)

In [82]:
from tensorflow.keras.models import load_model

best=load_model('./model4/347-8.2445-0.7619.hdf5')

pre = best.predict(X_real_ex)



고생1


In [223]:
#기존에 학습데이터와 후에 들어올 새로운 데이터의 예측값을 비교하기위 해 predict를 사용하여 변수화
pre_ex = best.predict(x_ex)

#새로 들어온 데이터 예측값
b_ex = best.predict(X_real_ex)


In [263]:
b0 = b_ex[0]
b1 = b_ex[1]

for i in range(30):
    for j in range(30):
        if (pre_ex[i][j]/b0[j]).round() == 1:
            

0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
17
18
18
18
18
18
18
18
18
18
19
19
19
19
19
19
19
19
19
19
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
21
21
21
21
21
21
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
23
23
23
24
24
2

In [251]:
print(b_ex)

[[ 0.66495955  0.5960745  -0.09068552 -0.13252155 -0.3187022  -0.05095164
  -0.18578759 -0.2729661  -0.16558182 -0.21302947 -0.1468511   0.08035976
  -0.36074615 -0.06869741 -0.1965251  -0.27323735 -0.18841591 -0.09469686
  -0.20926294 -0.06343122 -0.09985084 -0.02726909 -0.027835    4.054705
  -0.25826538 -0.12090027 -0.16812046 -0.04317562 -0.03216708 -0.06327869]
 [-0.19949062 -0.19873752 -0.09407944 -0.1412628  -0.31599128  0.01967904
  -0.18743964 -0.27335283 -0.14031474 -0.20912707 -0.14560716 -0.08672564
   2.9908848  -0.20787267 -0.05248486 -0.26445094 -0.13357942 -0.15678918
  -0.17852671 -0.05104877 -0.12655227 -0.02851646  0.03477922 -0.07763653
  -0.21434852 -0.07907297  0.3801695  -0.06756543 -0.0159988  -0.06067036]]


In [273]:
np.sum((b_ex[1] - pre_ex[5])**2)

8.882131e-14

In [215]:
pre_ex[5] #0~29   

array([-0.19949062, -0.19873752, -0.09407943, -0.1412628 , -0.31599125,
        0.01967904, -0.18743964, -0.27335283, -0.14031471, -0.20912707,
       -0.14560714, -0.08672564,  2.9908848 , -0.20787267, -0.052485  ,
       -0.26445094, -0.13357939, -0.15678914, -0.17852671, -0.05104876,
       -0.12655227, -0.02851646,  0.03477927, -0.07763653, -0.21434852,
       -0.07907293,  0.38016975, -0.06756543, -0.01599878, -0.06067035],
      dtype=float32)

In [216]:
b_ex[1] #0,1 // #[0] = 고생3 // [1] = 했어3

array([-0.19949062, -0.19873752, -0.09407944, -0.1412628 , -0.31599128,
        0.01967904, -0.18743964, -0.27335283, -0.14031474, -0.20912707,
       -0.14560716, -0.08672564,  2.9908848 , -0.20787267, -0.05248486,
       -0.26445094, -0.13357942, -0.15678918, -0.17852671, -0.05104877,
       -0.12655227, -0.02851646,  0.03477922, -0.07763653, -0.21434852,
       -0.07907297,  0.3801695 , -0.06756543, -0.0159988 , -0.06067036],
      dtype=float32)

In [221]:
y_en_np = np.array(y_en)

y_en
    
for i in range(len(pre)):
    for j in range(len(pre)):
        if not y_en.iloc[i][j] == 0:
            y_name = y_en.iloc[:,j].name
            print(y_name)

고생1


In [ ]:
for i in range(len(pre)):
    for j in range(len(pre)):
        if not y_en.iloc[i][j] == 0:
            y_name = y_en.iloc[:,j].name
            print(y_name)